In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
# Read in Data and view it: this dataframe will be updated to a new complete one made in SQL
Cause_df = pd.read_csv("../raw_data/merge_tables_output.csv")
Cause_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../raw_data/merge_tables_output.csv'

In [ ]:
pop_df.head()

In [ ]:
# Attempt to plot
Cause_df.plot()

In [ ]:
# Encode Labels for SciKit Learn
le = LabelEncoder()
df2=Cause_df.copy()
df2["Entity"] = le.fit_transform(df2["Entity"])
df2["Code"] = le.fit_transform(df2["Code"])
# View Results (Bottom Half)
df2.tail()

In [ ]:
# "Encode Age_Group to binary numbers"
df2_binary_encoded = pd.get_dummies(df2, columns=["Age_Group"])
df2_binary_encoded.head()


In [ ]:
# Begin Scaling Process
data_scaler = StandardScaler()
df2_scaled = data_scaler.fit_transform(df2_binary_encoded)
df2_scaled

In [ ]:
# Verify Means and Standard Deviations of each Column
for loop in df2_scaled:
    for y in loop:
        print(np.mean(df2_scaled[:,]))
        print(np.std(df2_scaled[:,]))


    
    

In [ ]:
#Double Check random Column Mean/STD
print(np.mean(df2_scaled[:,7]))
print(np.std(df2_scaled[:,7]))
    

In [ ]:
df2_binary_encoded['Max']=df2_binary_encoded[['Neoplasms','Tuberculosis','HIV/AIDS','Drowning','Malaria','Parkinsons_disease','Self']].apply(np.max,axis=1)
df2_binary_encoded.head()

In [ ]:
X=df2_binary_encoded.copy()
X = X.drop("Max", axis=1)
X.head()

In [ ]:
y = df2_binary_encoded["Max"].values
y[:5]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=78, train_size=0.80)
print(X_train2.shape)
print(X_test2.shape)
print(y_train2.shape)
print(y_test2.shape)

In [ ]:
scaler=StandardScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [4]:
model = tree.DecisionTreeClassifier()
model = model.fit(X_train_scaled, y_train)
predictions = model.predict(X_test_scaled)

NameError: name 'X_train_scaled' is not defined

In [3]:
predictions

NameError: name 'predictions' is not defined

In [ ]:
#Confusion matrix
cm = confusion_matrix(y_test, predictions)

cm_df = pd.Dataframe(cm, index=["Actual Max"], columns=["Predicted Max"])

cm_df